<a href="https://colab.research.google.com/github/QuakFooLee/python/blob/main/StockRecommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yahoo_fin

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=0a0e34a3d952ac415d2629ec0c620f343b6f32c93b3da6d11404141d9f06881b
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Load the list of stocks from the Excel file
stock_list_path = r"StockList.xlsx"
stocks_df = pd.read_excel(stock_list_path)

# Define the date threshold for the last 90 days
date_threshold = datetime.now() - timedelta(days=90)

# Define a function to fetch analyst recommendations using 'upgrades_downgrades'
def get_analyst_recommendations(stock_ticker):
    try:
        stock = yf.Ticker(stock_ticker)
        recommendations = stock.upgrades_downgrades
        if recommendations is not None:
            print(f"Columns available in {stock_ticker} recommendations: {recommendations.columns}")
            recommendations = recommendations.reset_index()
            recommendations['Stock'] = stock_ticker

            # Attempt to filter by date, if available
            if 'Date' in recommendations.columns:
                recommendations['Date'] = recommendations['Date'].apply(lambda x: x.date() if isinstance(x, datetime) else x)
                recommendations = recommendations[recommendations['Date'] >= date_threshold.date()]
            return recommendations
        else:
            print(f"No recommendations found for {stock_ticker}")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing {stock_ticker}: {e}")
        return pd.DataFrame()

# Create an empty DataFrame to hold all recommendations
all_recommendations = pd.DataFrame()

# Loop through each stock in the list and fetch its recommendations
for index, row in stocks_df.iterrows():
    stock_ticker = row['QuoteSymbol']
    stock_recommendations = get_analyst_recommendations(stock_ticker)
    all_recommendations = pd.concat([all_recommendations, stock_recommendations])


# Save the combined DataFrame to an Excel file
output_file_path = r'StockResults.xlsx'
all_recommendations.to_excel(output_file_path, index=False)

# Display the result
print(f"Total recommendations fetched: {len(all_recommendations)}")
all_recommendations

FileNotFoundError: [Errno 2] No such file or directory: 'StockList.xlsx'